In [ ]:
# imports

import os
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm.notebook import tqdm
from agents.evaluator import evaluate
from agents.items import Item
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection, Opportunity, Deal
import requests
import json
from agents.scanner_agent import ScannerAgent
import gradio as gr
from deal_agent_framework import DealAgentFramework

# environment

load_dotenv(override=True)
DB = "products_vectorstore"
openai = OpenAI()
MODEL = 'gpt-5-mini'


In [15]:
hf_token = os.environ['HF_TOKEN']
login(token=hf_token, add_to_git_credential=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [16]:
username = "ed-donner"
dataset = f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

README.md:   0%|          | 0.00/744 [00:00<?, ?B/s]

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\datasets--ed-donner--items_full. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to reg

train-00000-of-00001.parquet:   0%|          | 0.00/243M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


validation-00000-of-00001.parquet:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/800000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Loaded 800,000 training items, 10,000 validation items, 10,000 test items


In [17]:
# Creating a Chroma Database

client = chromadb.PersistentClient(path=DB)

In [18]:
# Introducing the SentenceTransformer Encoding LLM

encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [20]:
# Check if the collection exists; if not, create it

collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]

if collection_name not in existing_collection_names:
    collection = client.create_collection(collection_name)
    for i in tqdm(range(0, len(train), 1000)):
        documents = [item.summary for item in train[i: i+1000]]
        vectors = encoder.encode(documents).astype(float).tolist()
        metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
        ids = [f"doc_{j}" for j in range(i, i+1000)]
        ids = ids[:len(documents)]
        collection.add(ids=ids, documents=documents, embeddings=vectors, metadatas=metadatas)

collection = client.get_or_create_collection(collection_name)

  0%|          | 0/800 [00:00<?, ?it/s]

In [21]:
def vector(item):
    return encoder.encode(item.summary)

def find_similars(item):
    vec = vector(item)
    results = collection.query(query_embeddings=vec.astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [22]:
find_similars(test[0])

(['Title: Old Blood Noise Endeavors Procession Reverb  \nCategory: Audio Effects  \nBrand: Old Blood Noise Endeavors  \nDescription: A compact, sci‑fi inspired reverb pedal with three modulation modes for creating otherworldly echo effects.  \nDetails: Features adjustable mix, decay, speed, depth, and footswitches for bypass and hold; powered by 9\u202fV DC with 60\u202fmA draw.',
  'Title: Boss MD‑2 Mega Distortion Modulation Multi‑Effects Pedal  \nCategory: Music Equipment → Effects Pedals  \nBrand: Boss  \nDescription: A powerful distortion pedal that delivers extreme low‑end crunch and endless sustain for metal and hard rock.  \nDetails: Features a Gain Boost circuit, bottom‑heavy Bottom control for 6/7‑string guitars, adjustable Tone knob, and 1\u202fMΩ input impedance.',
  'Title: Old Blood Noise Endeavors Mondegreen Delay Pedal  \nCategory: Musical Instruments / Effects Pedals  \nBrand: Old Blood Noise  \nDescription: A digital delay pedal that transforms your signal into creati

In [23]:
# We need to give some context to GPT-5.1 by selecting 5 products with similar descriptions

def make_context(similars, prices):
    message = "For context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

documents, prices = find_similars(test[0])
print(make_context(documents, prices))


For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Old Blood Noise Endeavors Procession Reverb  
Category: Audio Effects  
Brand: Old Blood Noise Endeavors  
Description: A compact, sci‑fi inspired reverb pedal with three modulation modes for creating otherworldly echo effects.  
Details: Features adjustable mix, decay, speed, depth, and footswitches for bypass and hold; powered by 9 V DC with 60 mA draw.
Price is $209.00

Potentially related product:
Title: Boss MD‑2 Mega Distortion Modulation Multi‑Effects Pedal  
Category: Music Equipment → Effects Pedals  
Brand: Boss  
Description: A powerful distortion pedal that delivers extreme low‑end crunch and endless sustain for metal and hard rock.  
Details: Features a Gain Boost circuit, bottom‑heavy Bottom control for 6/7‑string guitars, adjustable Tone knob, and 1 MΩ input impedance.
Price is $109.99

Potentially related product:
Title: Old Blood Noise End

In [24]:
def messages_for(item, similars, prices):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}\n\n"
    message += make_context(similars, prices)
    return [{"role": "user", "content": message}]

documents, prices = find_similars(test[0])
print(messages_for(test[0], documents, prices)[0]['content'])

Estimate the price of this product. Respond with the price, no explanation

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Old Blood Noise Endeavors Procession Reverb  
Category: Audio Effects  
Brand: Old Blood Noise Endeavors  
Description: A compact, sci‑fi inspired reverb pedal with three modulation modes for creating otherworldly echo effects.  
Details: Features adjustable mix, decay, speed, depth, and footswitches for bypass and hold; powered by 9 V 

In [25]:
# The function for gpt-5-mini

def gpt_5__1_rag(item):
    documents, prices = find_similars(item)
    response = completion(model="gpt-5.1", messages=messages_for(item, documents, prices), reasoning_effort="none", seed=42)
    return response.choices[0].message.content

In [26]:
gpt_5__1_rag(test[0])

'$229.00'

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$229.00'...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [27]:
evaluate(gpt_5__1_rag, test)

  0%|          | 0/200 [00:00<?, ?it/s]

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$39.95',...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$46.49',...: None}, annotations=[]), input_type=Mess

$10 $24 $15 $23 $30 $130 

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$164.95'...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$756.00'...: None}, annotations=[]), input_type=Mess

$21 

...

$56 $9 $61 $15 $150 

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$150.00'...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$38.99',...: None}, annotations=[]), input_type=Mess

$29 $20 $17 $33 $12 $20 $2 $10 $6 $8 $31 $0 $5 

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='272.41',...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$17.49',...: None}, annotations=[]), input_type=Mess

$21 $8 $1 $8 

In [28]:
import modal
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()

def specialist(item):
    return pricer.price.remote(item.summary)

def get_price(reply):
    reply = reply.replace("$", "").replace(",", "")
    match = re.search(r"[-+]?\d*\.\d+|\d+", reply)
    return float(match.group()) if match else 0

In [29]:
# Deep Neural Network

from agents.deep_neural_network import DeepNeuralNetworkInference

runner = DeepNeuralNetworkInference()
runner.setup()
runner.load("deep_neural_network.pth")

def deep_neural_network(item):
    return runner.inference(item.summary)

In [30]:
def ensemble(item):
    price1 = get_price(gpt_5__1_rag(item))
    price2 = specialist(item)
    price3 = deep_neural_network(item)
    return price1 * 0.8 + price2 * 0.1 + price3 * 0.1


In [31]:
evaluate(ensemble, test)

  0%|          | 0/200 [00:00<?, ?it/s]

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='59.85', ...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$39.95',...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(E

$14 $32 

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$99.99',...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$164.95'...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(E

$15 $15 $26 $130 $6 

...

$114 $23 $17 $16 $42 $9 $18 $0 

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='$23.99',...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='79.99', ...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(E

$9 $5 $8 $28 $1 $6 

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='49.99', ...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])

c:\Users\Admin\projects\llm_engineering\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='272.41',...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(E

$22 $6 $4 $5 

In [32]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [35]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 3/3 [01:58<00:00, 39.55s/it]


In [36]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [37]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [38]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Best Buy Android Phone Winter Deals: Up to $350 off + free shipping
Details: As part of the Best Buy's Winter Sale, get deals on android phones from brands like Samsung, Google, Motorola, and more. We've pictured Unlocked Motorola Moto G Power 5G 128GB Phone for $129.99 ($170 savings). My Best Buy members get free shipping. (It's free to join. Shipping is free for everyone over

In [39]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="minimal")
results = response.choices[0].message.parsed
results

DealSelection(deals=[Deal(product_description="Unlocked Motorola Moto G Power 5G with 128GB storage is an Android smartphone focused on long battery life and 5G connectivity. The model offers a midrange processor, ample internal storage for apps and media, and typical smartphone features like a multi-lens camera system and large IPS/AMOLED display suited for streaming and everyday use. It's presented as an unlocked device, so it can be used with most carriers that support GSM/5G bands.", price=129.99, url='https://www.dealnews.com/Best-Buy-Android-Phone-Winter-Deals-Up-to-350-off-free-shipping/21801832.html?iref=rss-c142'), Deal(product_description='JBL Tune 670NC are over-ear wireless headphones with active noise cancellation, Bluetooth connectivity, and a battery designed for extended listening sessions. They include on-ear controls for playback and calls, a foldable design for portability, and JBL audio tuning aimed at balanced sound with punchy bass for music, movies, and calls.', 

In [40]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


Unlocked Motorola Moto G Power 5G with 128GB storage is an Android smartphone focused on long battery life and 5G connectivity. The model offers a midrange processor, ample internal storage for apps and media, and typical smartphone features like a multi-lens camera system and large IPS/AMOLED display suited for streaming and everyday use. It's presented as an unlocked device, so it can be used with most carriers that support GSM/5G bands.
129.99
https://www.dealnews.com/Best-Buy-Android-Phone-Winter-Deals-Up-to-350-off-free-shipping/21801832.html?iref=rss-c142

JBL Tune 670NC are over-ear wireless headphones with active noise cancellation, Bluetooth connectivity, and a battery designed for extended listening sessions. They include on-ear controls for playback and calls, a foldable design for portability, and JBL audio tuning aimed at balanced sound with punchy bass for music, movies, and calls.
34.0
https://www.dealnews.com/JBL-Tune-670-NC-Active-Noise-Cancelling-Wireless-Headphones-f

In [41]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [42]:
agent = ScannerAgent()
result = agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 30 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


In [43]:
# 3 pretend functions

def scan_the_internet_for_bargains() -> str:
    """ This tool scans the internet for great deals and gets a curated list of promising deals """
    print("Fake function to scan the internet - this returns a hardcoded set of deals")
    return test_results.model_dump_json()

def estimate_true_value(description: str) -> str:
    """
    This tool estimates the true value of a product based on a text description of it
    """
    print(f"Fake function to estimating true value of {description[:20]}... - this always returns $300")
    return f"Product {description} has an estimated true value of $300"

def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float, url: str) -> str:
    """
    This tool notifies the user of a great deal, given a description of it, the price of the deal, and the estimated true value
    """
    print(f"Fake function to notify user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return "notification sent ok"

In [44]:
notify_user_of_deal("a new iphone", 100, 1000, "https://www.apple.com/iphone")

Fake function to notify user of a new iphone which costs 100 and estimate is 1000


'notification sent ok'

In [45]:
scan_function = {
        "name": "scan_the_internet_for_bargains",
        "description": "Returns top bargains scraped from the internet along with the price each item is being offered for",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": False
        }
    }

estimate_function = {
    "name": "estimate_true_value",
    "description": "Given the description of an item, estimate how much it is actually worth",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item to be estimated"
            },
        },
        "required": ["description"],
        "additionalProperties": False
    }
}

notify_function = {
    "name": "notify_user_of_deal",
    "description": "Send the user a push notification about the single most compelling deal; only call this one time",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item itself scraped from the internet"
            },
            "deal_price": {
                "type": "number",
                "description": "The price offered by this deal scraped from the internet"
            }
            ,
            "estimated_true_value": {
                "type": "number",
                "description": "The estimated actual value that this is worth"
            }
            ,
            "url": {
                "type": "string",
                "description": "The URL of this deal as scraped from the internet"
            }
        },
        "required": ["description", "deal_price", "estimated_true_value", "url"],
        "additionalProperties": False
    }
}

In [46]:
tools = [{"type": "function", "function": scan_function},
 {"type": "function", "function": estimate_function},
 {"type": "function", "function": notify_function}
 ]

In [47]:
def handle_tool_call(message):
    """
    Actually call the tools associated with this message
    """
    results = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [48]:
system_message = "You find great deals on bargain products using your tools, and notify the user of the best bargain."
user_message = """
First, use your tool to scan the internet for bargain deals. Then for each deal, use your tool to estimate its true value.
Then pick the single most compelling deal where the price is much lower than the estimated true value, and use your tool to notify the user.
Then just reply OK to indicate success.
"""
messages = [{"role": "system", "content": system_message},{"role": "user", "content": user_message}]

In [51]:
from pydantic import BaseModel
from typing import List

class Deal(BaseModel):
    title: str
    price: float
    url: str

class DealResults(BaseModel):
    deals: List[Deal]

test_results = DealResults(
    deals=[
        Deal(title="iPhone 13 Pro Max", price=699.0, url="https://example.com/1"),
        Deal(title="Samsung Galaxy S22", price=499.0, url="https://example.com/2"),
    ]
)


In [52]:
done = False
while not done:
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        results = handle_tool_call(message)
        messages.append(message)
        messages.extend(results)
    else:
        done = True
response.choices[0].message.content

Fake function to scan the internet - this returns a hardcoded set of deals
Fake function to estimating true value of iPhone 13 Pro Max (u... - this always returns $300
Fake function to estimating true value of Samsung Galaxy S22 l... - this always returns $300
Fake function to notify user of iPhone 13 Pro Max which costs 699 and estimate is 300


'OK'

In [53]:
# Autonomous Planning Function

root = logging.getLogger()
root.setLevel(logging.INFO)

In [54]:
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [55]:
agent.plan()

AttributeError: 'ScannerAgent' object has no attribute 'plan'

In [57]:
with gr.Blocks(title="The Price is Right", fill_width=True) as ui:

    initial_deal = Deal(product_description="Example description", price=100.0, url="https://cnn.com")
    initial_opportunity = Opportunity(deal=initial_deal, estimate=200.0, discount=100.0)
    opportunities = gr.State([initial_opportunity])

    def get_table(opps):
        return [[opp.deal.product_description, opp.deal.price, opp.estimate, opp.discount, opp.deal.url] for opp in opps]

    with gr.Row():
        gr.Markdown('<div style="text-align: center;font-size:24px">"The Price is Right" - Deal Hunting Agentic AI</div>')
    with gr.Row():
        gr.Markdown('<div style="text-align: center;font-size:14px">Deals surfaced so far:</div>')
    with gr.Row():
        opportunities_dataframe = gr.Dataframe(
            headers=["Description", "Price", "Estimate", "Discount", "URL"],
            wrap=True,
            column_widths=[4, 1, 1, 1, 2],
            row_count=10,
            col_count=5,
            max_height=400,
        )

    ui.load(get_table, inputs=[opportunities], outputs=[opportunities_dataframe])

ui.launch(inbrowser=True)

ValidationError: 1 validation error for Deal
title
  Field required [type=missing, input_value={'product_description': '...url': 'https://cnn.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [58]:
agent_framework = DealAgentFramework()
agent_framework.init_agents_as_needed()

with gr.Blocks(title="The Price is Right", fill_width=True) as ui:

    initial_deal = Deal(product_description="Example description", price=100.0, url="https://cnn.com")
    initial_opportunity = Opportunity(deal=initial_deal, estimate=200.0, discount=100.0)
    opportunities = gr.State([initial_opportunity])

    def get_table(opps):
        return [[opp.deal.product_description, opp.deal.price, opp.estimate, opp.discount, opp.deal.url] for opp in opps]

    def do_select(opportunities, selected_index: gr.SelectData):
        row = selected_index.index[0]
        opportunity = opportunities[row]
        agent_framework.planner.messenger.alert(opportunity)

    with gr.Row():
        gr.Markdown('<div style="text-align: center;font-size:24px">"The Price is Right" - Deal Hunting Agentic AI</div>')
    with gr.Row():
        gr.Markdown('<div style="text-align: center;font-size:14px">Deals surfaced so far:</div>')
    with gr.Row():
        opportunities_dataframe = gr.Dataframe(
            headers=["Description", "Price", "Estimate", "Discount", "URL"],
            wrap=True,
            column_widths=[4, 1, 1, 1, 2],
            row_count=10,
            col_count=5,
            max_height=400,
        )

    ui.load(get_table, inputs=[opportunities], outputs=[opportunities_dataframe])
    opportunities_dataframe.select(do_select, inputs=[opportunities], outputs=[])

ui.launch(inbrowser=True)

INFO:root:[Agent Framework] Initializing Agent Framework


[2026-01-12 19:42:33 +0530] [Agents] [INFO] [Agent Framework] Initializing Agent Framework


INFO:root:[Planning Agent] Planning Agent is initializing


[2026-01-12 19:42:33 +0530] [Agents] [INFO] [Planning Agent] Planning Agent is initializing


INFO:root:[Scanner Agent] Scanner Agent is initializing


...

INFO:root:[Planning Agent] Planning Agent is ready


[2026-01-12 19:42:41 +0530] [Agents] [INFO] [Planning Agent] Planning Agent is ready


INFO:root:[Agent Framework] Agent Framework is ready


[2026-01-12 19:42:41 +0530] [Agents] [INFO] [Agent Framework] Agent Framework is ready


ValidationError: 1 validation error for Deal
title
  Field required [type=missing, input_value={'product_description': '...url': 'https://cnn.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [ ]:
# Reset memory back to 2 deals discovered in the past

from deal_agent_framework import DealAgentFramework
DealAgentFramework.reset_memory()

In [ ]:
import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
!uv run price_is_right.py